# Uncertainty Principle

This notebook contains the programmatic verification for the **Uncertainty Principle** entry from the THEORIA dataset.

**Entry ID:** uncertainty_principle  
**Required Library:** sympy 1.13.1

## Description
The Heisenberg uncertainty principle states that for any pair of non-commuting observables their statistical spreads (standard deviations) `Delta A` and `Delta B` in a given quantum state cannot both be made arbitrarily small. The general Robertson relation ties the product `Delta A * Delta B` to the expectation value of their commutator, while canonical coordinate-momentum pairs `(q_i, p_i)` satisfy `Delta q_i * Delta p_i >= hbar/2`. For a single spatial coordinate `x` and its conjugate momentum `p_x` this reduces to the familiar `Delta x * Delta p_x >= hbar/2`. The limitation comes from the non-commuting operator algebra rather than experimental imperfections.

## Installation
First, let's install the required library:

In [ ]:
# Install required library with exact version
!pip install sympy==1.13.1

## Programmatic Verification

The following code verifies the derivation mathematically:

In [ ]:
import sympy as sp

# ============================================================
# Programmatic verification for Heisenberg uncertainty entry
# We mirror the conceptual derivation steps 1-15 using a
# concrete 2D example with A = sigma_x, B = sigma_y.
# ============================================================

# Step 0: symbols and generic normalized qubit state |:psi:)
theta, phi = sp.symbols('theta phi', real=True)
psi = sp.Matrix([
    sp.cos(theta/2),
    sp.exp(sp.I*phi) * sp.sin(theta/2)
])

# Step 0b: Pauli matrices and identity (Hermitian test case for A, B)
sigma_x = sp.Matrix([[0, 1], [1, 0]])
sigma_y = sp.Matrix([[0, -sp.I], [sp.I, 0]])
sigma_z = sp.Matrix([[1, 0], [0, -1]])
I2 = sp.eye(2)
A = sigma_x
B = sigma_y

# Sanity check: A and B are Hermitian and squares are identity
assert A.H == A
assert B.H == B
assert A**2 == I2
assert B**2 == I2

# ------------------------------------------------------------
# Step 1 (derivation step 1): expectation values a_0, b_0
# ------------------------------------------------------------
a0 = (psi.H * A * psi)[0]
b0 = (psi.H * B * psi)[0]
a0_s = sp.simplify(a0)
b0_s = sp.simplify(b0)

# Check that <A> and <B> are real for real theta, phi
assert sp.simplify(a0_s - sp.conjugate(a0_s)) == 0
assert sp.simplify(b0_s - sp.conjugate(b0_s)) == 0

# ------------------------------------------------------------
# Steps 2-4 (derivation): variances and operator form
# Here we use that for Pauli matrices A^2 = B^2 = I, so
#   DeltaA^2 = 1 - a_0^2,  DeltaB^2 = 1 - b_0^2,
# which matches the operator expression DeltaA^2 = <(A - a_0 I)^2>.
# ------------------------------------------------------------
DeltaA2 = sp.simplify(1 - a0_s**2)
DeltaB2 = sp.simplify(1 - b0_s**2)

# Compute the same variances directly from the centered operators
A_prime = A - a0_s * I2
B_prime = B - b0_s * I2
DeltaA2_from_op = sp.trigsimp(sp.simplify((psi.H * (A_prime**2) * psi)[0]))
DeltaB2_from_op = sp.trigsimp(sp.simplify((psi.H * (B_prime**2) * psi)[0]))

# Check that the operator expression reproduces the same variances
assert sp.simplify(sp.expand(DeltaA2_from_op - DeltaA2, complex=True)) == 0
assert sp.simplify(sp.expand(DeltaB2_from_op - DeltaB2, complex=True)) == 0

# ------------------------------------------------------------
# Steps 5-6 (derivation): define |:alpha:), |:beta:) and norms
# ------------------------------------------------------------
alpha = A_prime * psi   # |:alpha:) = A'|:psi:)
beta  = B_prime * psi   # |:beta:)  = B'|:psi:)

DeltaA2_from_alpha = sp.simplify((alpha.H * alpha)[0])
DeltaB2_from_beta  = sp.simplify((beta.H * beta)[0])

# Check ||alpha||^2 = DeltaA^2 and ||beta||^2 = DeltaB^2
assert sp.simplify(sp.expand(DeltaA2_from_alpha - DeltaA2, complex=True)) == 0
assert sp.simplify(sp.expand(DeltaB2_from_beta  - DeltaB2, complex=True)) == 0

# ------------------------------------------------------------
# Step 7 (derivation): Cauchy-Schwarz inequality
# |<:alpha:|:beta:>|^2 <= ||alpha||^2 ||beta||^2
# ------------------------------------------------------------
z = (alpha.H * beta)[0]   # z = (:alpha:|:beta:)

lhs_cs = sp.simplify(DeltaA2 * DeltaB2)
rhs_cs = sp.simplify(z * sp.conjugate(z))
diff_cs = sp.simplify(lhs_cs - rhs_cs)

# The Cauchy-Schwarz inequality is satisfied (diff_cs >= 0)
# Verify non-negativity numerically at test points
for theta_val in [0, sp.pi/6, sp.pi/4, sp.pi/3, sp.pi/2]:
    for phi_val in [0, sp.pi/6, sp.pi/4, sp.pi/3, sp.pi/2]:
        val = complex(diff_cs.subs({theta: theta_val, phi: phi_val}))
        assert val.real >= -1e-10  # Allow small numerical error
        assert abs(val.imag) < 1e-10  # Should be real

# ------------------------------------------------------------
# Steps 8-10 (derivation): anticommutator and commutator
# A' B' = 1/2 {A', B'} + 1/2 [A', B'] and decomposition of z
# ------------------------------------------------------------
anticom = A_prime * B_prime + B_prime * A_prime    # {A',B'}
comm   = A_prime * B_prime - B_prime * A_prime    # [A',B']

# Rebuild z from anticommutator + commutator pieces
z_from_anticom_comm = sp.simplify(
    sp.Rational(1, 2) * (psi.H * anticom * psi)[0]
    + sp.Rational(1, 2) * (psi.H * comm * psi)[0]
)
assert sp.simplify(sp.expand(z_from_anticom_comm - z, complex=True)) == 0

# Real and imaginary parts from anticommutator and commutator
Re_z = sp.simplify(sp.re(z))
Im_z = sp.simplify(sp.im(z))

Re_z_from_anticom = sp.simplify(sp.Rational(1, 2) * (psi.H * anticom * psi)[0])
Im_z_from_comm    = sp.simplify((psi.H * comm * psi)[0] / (2*sp.I))

assert sp.simplify(sp.expand(Re_z - Re_z_from_anticom, complex=True)) == 0
assert sp.simplify(sp.expand(Im_z - Im_z_from_comm, complex=True)) == 0

# ------------------------------------------------------------
# Step 11 (derivation): [A', B'] = [A, B]
# ------------------------------------------------------------
comm_Ap_Bp = A_prime * B_prime - B_prime * A_prime
comm_A_B   = A * B - B * A
assert sp.simplify(comm_Ap_Bp - comm_A_B) == sp.zeros(2, 2)

# ------------------------------------------------------------
# Steps 12-13 (derivation): Robertson inequality
# (DeltaA)^2 (DeltaB)^2 >= (1/4) |< [A,B] >|^2
# ------------------------------------------------------------
comm_exp = sp.simplify((psi.H * comm_A_B * psi)[0])

lhs_uncert = sp.simplify(DeltaA2 * DeltaB2)
rhs_uncert = sp.simplify((comm_exp * sp.conjugate(comm_exp)) / 4)
diff_uncert = sp.simplify(lhs_uncert - rhs_uncert)

# The Robertson inequality is satisfied (diff_uncert >= 0)
# Verify non-negativity numerically at test points
for theta_val in [0, sp.pi/6, sp.pi/4, sp.pi/3, sp.pi/2]:
    for phi_val in [0, sp.pi/6, sp.pi/4, sp.pi/3, sp.pi/2]:
        val = complex(diff_uncert.subs({theta: theta_val, phi: phi_val}))
        assert val.real >= -1e-10  # Allow small numerical error
        assert abs(val.imag) < 1e-10  # Should be real

# ------------------------------------------------------------
# Steps 14-15 (derivation): abstract canonical pair (q_i, p_i)
# and Heisenberg bound Delta q_i Delta p_i >= hbar / 2
# ------------------------------------------------------------
Dq, Dp, hbar = sp.symbols('Delta_q Delta_p hbar', positive=True, real=True)

# Canonical commutation relations imply <[q_i, p_i]> = i hbar in any state
comm_qp_expect = sp.I * hbar

# Robertson bound for this pair: Delta q_i Delta p_i >= (1/2) |< [q_i, p_i] >|
rhs_canonical = sp.simplify(sp.sqrt(comm_qp_expect * sp.conjugate(comm_qp_expect)) / 2)

# Check the algebraic form of the lower bound: |i hbar| / 2 = hbar / 2
assert sp.simplify(rhs_canonical - hbar/2) == 0

print('Programmatic checks for the Heisenberg uncertainty derivation completed successfully.')


## Source

📖 **View this entry:** [theoria-dataset.org/entries.html?entry=uncertainty_principle.json](https://theoria-dataset.org/entries.html?entry=uncertainty_principle.json)

This verification code is part of the [THEORIA dataset](https://github.com/theoria-dataset/theoria-dataset), a curated collection of theoretical physics derivations with programmatic verification.

**License:** CC-BY 4.0